In [1]:
# 식품섭취조사_7

import olefile
import zlib
import struct

def get_hwp_text(filename) :
    f = olefile.OleFileIO(filename)
    dirs = f.listdir()

    # HWP 파일 검증
    if ["FileHeader"] not in dirs or \
       ["\x05HwpSummaryInformation"] not in dirs :
        raise Exception("Not Valid HWP.")

    # 문서 포맷 압축 여부 확인
    header = f.openstream("FileHeader")
    header_data = header.read()
    is_compressed = (header_data[36] & 1) == 1

    # Body Sections 불러오기
    nums = []
    for d in dirs :
        if d[0] == "BodyText" :
            nums.append(int(d[1][len("Section"):]))
    sections = ["BodyText/Section" + str(x) for x in sorted(nums)]

    # 전체 text 추출
    text = ""
    for section in sections :
        bodytext = f.openstream(section)
        data = bodytext.read()
        if is_compressed :
            unpacked_data = zlib.decompress(data, -15)
        else:
            unpacked_data = data
    
        # 각 Section 내 text 추출    
        section_text = ""
        i = 0
        size = len(unpacked_data)
        while i < size :
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in [67] and header != 52429891 and header != 18875459 and  header != 102761539 :
                rec_data = unpacked_data[i + 4 : i + 4 + rec_len]
                print(header)
                print(rec_data.decode('UTF-16'))
                section_text += rec_data.decode('UTF-16')
                section_text += "\n"

            i += 4 + rec_len
        
        text += section_text
        text += "\n"

    return text

In [2]:
# 한자 적힌 거
'''
52429891
捤獥汤捯
氠瑢


18875459

氠瑢


18875459

氠瑢


18875459

氠瑢


18875459

氠瑢


18875459

氠瑢


18875459

氠瑢


102761539

氠瑢

氠瑢
桤灧桤灧桤灧桤灧
'''

'\n52429891\n\x02捤獥\x02\x02汤捯\x02\x0b\n氠瑢\x0b\n\n\n18875459\n\x0b\n氠瑢\x0b\n\n\n18875459\n\x0b\n氠瑢\x0b\n\n\n18875459\n\x0b\n氠瑢\x0b\n\n\n18875459\n\x0b\n氠瑢\x0b\n\n\n18875459\n\x0b\n氠瑢\x0b\n\n\n18875459\n\x0b\n氠瑢\x0b\n\n\n102761539\n\x0b\n氠瑢\x0b\n\x0b\n氠瑢\x0b\n\x15桤灧\x15\x15桤灧\x15\x15桤灧\x15\x15桤灧\x15\n'

In [3]:
# HWP 파일 불러오기

abc7 = get_hwp_text('식품섭취조사_7.hwp')
abc7

14683203
N_DIET
14683203
식사요법여부
23071811
N_DIET_WHY
14683203
식사요법이유
16780355
N_DT_DS
20974659
식사요법이유_질환
18877507
N_DT_ETC
20974659
식사요법이유_기타
18877507
N_DUSUAL
20974659
평소식사량과 비교
16780355
N_WAT_C
16780355
물섭취량(컵)
12586051
N_PRG
35654723
임신 및 수유여부(가임기여성)
16780355
N_BFD_Y
29363267
모유수유여부(만1~3세)
12586051
N_DAY
14683203
영양조사요일
14683203
N_MEAL
10488899
끼니구분
14683203
N_MEAL
10488899
끼니구분
18877507
N_MEAL_T
10488899
식사시간
18877507
N_MEAL_P
10488899
식사장소
16780355
N_MTYPE
10488899
매식여부
18877507
N_MEAL_W
16780355
타인동반 여부
16780355
N_DNAME
8391747
음식명
16780355
N_DCODE
10488899
음식코드
18877507
N_CD_VOL
27266115
음식의 조리총량(부피)
16780355
N_CD_WT
27266115
음식의 조리총량(중량)
18877507
N_TD_VOL
25168963
음식의 섭취량(부피)
16780355
N_TD_WT
25168963
음식의 섭취량(중량)
16780355
N_FCODE
10488899
식품코드
16780355
N_FNAME
20974659
식품코드명(한글)
16780355
N_ENAME
20974659
식품코드명(영문)
16780355
N_FD_NS
23071811
식품명 및 식품상태
16780355
N_FM_WT
20974659
식품재료량(중량)
20974659
N_PRODUCT
8391747
제품명
16780355
N_CNAME
12586051
제조회사명
10488899
N_AP
16780355
동식물성 구분


'N_DIET\r\n식사요법여부\r\nN_DIET_WHY\r\n식사요법이유\r\nN_DT_DS\r\n식사요법이유_질환\r\nN_DT_ETC\r\n식사요법이유_기타\r\nN_DUSUAL\r\n평소식사량과 비교\r\nN_WAT_C\r\n물섭취량(컵)\r\nN_PRG\r\n임신 및 수유여부(가임기여성)\r\nN_BFD_Y\r\n모유수유여부(만1~3세)\r\nN_DAY\r\n영양조사요일\r\nN_MEAL\r\n끼니구분\r\nN_MEAL\r\n끼니구분\r\nN_MEAL_T\r\n식사시간\r\nN_MEAL_P\r\n식사장소\r\nN_MTYPE\r\n매식여부\r\nN_MEAL_W\r\n타인동반 여부\r\nN_DNAME\r\n음식명\r\nN_DCODE\r\n음식코드\r\nN_CD_VOL\r\n음식의 조리총량(부피)\r\nN_CD_WT\r\n음식의 조리총량(중량)\r\nN_TD_VOL\r\n음식의 섭취량(부피)\r\nN_TD_WT\r\n음식의 섭취량(중량)\r\nN_FCODE\r\n식품코드\r\nN_FNAME\r\n식품코드명(한글)\r\nN_ENAME\r\n식품코드명(영문)\r\nN_FD_NS\r\n식품명 및 식품상태\r\nN_FM_WT\r\n식품재료량(중량)\r\nN_PRODUCT\r\n제품명\r\nN_CNAME\r\n제조회사명\r\nN_AP\r\n동식물성 구분\r\nN_KINDG1\r\n식품군 분류1\r\nN_KINDG2\r\n식품군 분류2\r\nN_FCODE3\r\n3차 식품코드\r\nN_FNAME3\r\n3차 식품코드명\r\nNF_INTK\r\n식품섭취량\r\nNF_INTK3\r\n3차 식품코드용 식품섭취량\r\nNF_EN\r\n에너지\r\nNF_WATER\r\n수분\r\nNF_PROT\r\n단백질\r\nNF_FAT\r\n지방\r\nNF_SFA\r\n포화지방산\r\nNF_MUFA\r\n단일불포화지방산\r\nNF_PUFA\r\n다가불포화지방산\r\nNF_N3\r\nn-3계 지방산\r\nNF_N6\r\nn-6계 지방산\r\nNF_10_0\r\n카프르산\r\nNF_12_0\

In [4]:
# ''\r\n' split

split_abc7 = abc7.split('\r\n')
split_abc7

['N_DIET',
 '식사요법여부',
 'N_DIET_WHY',
 '식사요법이유',
 'N_DT_DS',
 '식사요법이유_질환',
 'N_DT_ETC',
 '식사요법이유_기타',
 'N_DUSUAL',
 '평소식사량과 비교',
 'N_WAT_C',
 '물섭취량(컵)',
 'N_PRG',
 '임신 및 수유여부(가임기여성)',
 'N_BFD_Y',
 '모유수유여부(만1~3세)',
 'N_DAY',
 '영양조사요일',
 'N_MEAL',
 '끼니구분',
 'N_MEAL',
 '끼니구분',
 'N_MEAL_T',
 '식사시간',
 'N_MEAL_P',
 '식사장소',
 'N_MTYPE',
 '매식여부',
 'N_MEAL_W',
 '타인동반 여부',
 'N_DNAME',
 '음식명',
 'N_DCODE',
 '음식코드',
 'N_CD_VOL',
 '음식의 조리총량(부피)',
 'N_CD_WT',
 '음식의 조리총량(중량)',
 'N_TD_VOL',
 '음식의 섭취량(부피)',
 'N_TD_WT',
 '음식의 섭취량(중량)',
 'N_FCODE',
 '식품코드',
 'N_FNAME',
 '식품코드명(한글)',
 'N_ENAME',
 '식품코드명(영문)',
 'N_FD_NS',
 '식품명 및 식품상태',
 'N_FM_WT',
 '식품재료량(중량)',
 'N_PRODUCT',
 '제품명',
 'N_CNAME',
 '제조회사명',
 'N_AP',
 '동식물성 구분',
 'N_KINDG1',
 '식품군 분류1',
 'N_KINDG2',
 '식품군 분류2',
 'N_FCODE3',
 '3차 식품코드',
 'N_FNAME3',
 '3차 식품코드명',
 'NF_INTK',
 '식품섭취량',
 'NF_INTK3',
 '3차 식품코드용 식품섭취량',
 'NF_EN',
 '에너지',
 'NF_WATER',
 '수분',
 'NF_PROT',
 '단백질',
 'NF_FAT',
 '지방',
 'NF_SFA',
 '포화지방산',
 'NF_MUFA',
 '단일불포화지방산',
 'NF_PUFA',

In [5]:
## 딕셔너리 형태로 생성

abc7_dic = dict()
for i in range(0, len(split_abc7)-1,2) :
    abc7_dic[split_abc7[i]] = split_abc7[i+1] 

print(abc7_dic)

{'N_DIET': '식사요법여부', 'N_DIET_WHY': '식사요법이유', 'N_DT_DS': '식사요법이유_질환', 'N_DT_ETC': '식사요법이유_기타', 'N_DUSUAL': '평소식사량과 비교', 'N_WAT_C': '물섭취량(컵)', 'N_PRG': '임신 및 수유여부(가임기여성)', 'N_BFD_Y': '모유수유여부(영유아)', 'N_DAY': '영양조사요일', 'N_MEAL': '끼니구분', 'N_MEAL_T': '식사시간', 'N_MEAL_P': '식사장소', 'N_MTYPE': '매식여부', 'N_MEAL_W': '타인동반 여부', 'N_DNAME': '음식명', 'N_DCODE': '음식코드', 'N_CD_VOL': '음식의 조리총량(부피)', 'N_CD_WT': '음식의 조리총량(중량)', 'N_TD_VOL': '음식의 섭취량(부피)', 'N_TD_WT': '음식의 섭취량(중량)', 'N_FCODE': '식품코드', 'N_FNAME': '식품코드명(한글)', 'N_ENAME': '식품코드명(영문)', 'N_FD_NS': '식품명 및 식품상태', 'N_FM_WT': '식품재료량(중량)', 'N_PRODUCT': '제품명', 'N_CNAME': '제조회사명', 'N_AP': '동식물성 구분', 'N_KINDG1': '식품군 분류1', 'N_KINDG2': '식품군 분류2', 'N_FCODE3': '3차 식품코드', 'N_FNAME3': '3차 식품코드명', 'NF_INTK': '식품섭취량', 'NF_INTK3': '3차 식품코드용 식품섭취량', 'NF_EN': '에너지', 'NF_WATER': '수분', 'NF_PROT': '단백질', 'NF_FAT': '지방', 'NF_SFA': '포화지방산', 'NF_MUFA': '단일불포화지방산', 'NF_PUFA': '다가불포화지방산', 'NF_N3': 'n-3계 지방산', 'NF_N6': 'n-6계 지방산', 'NF_10_0': '카프르산', 'NF_12_0': '라우르산', 'NF_14_0'

In [6]:
# 데이터프레임 생성

import pandas as pd

abc7_df = pd.DataFrame(list(abc7_dic.items()),
                   columns = ['변수명','변수설명'])

abc7_df

,변수명,변수설명
0,N_DIET,식사요법여부
1,N_DIET_WHY,식사요법이유
2,N_DT_DS,식사요법이유_질환
3,N_DT_ETC,식사요법이유_기타
4,N_DUSUAL,평소식사량과 비교
...,...,...
142,HEI_NA,나트륨 섭취
143,HEI_SWEET,"당류, 음료류의 에너지섭취비율"
144,HEI_CHO,탄수화물 에너지섭취비율
145,HEI_FAT,지방 에너지섭취비율


In [7]:
## 데이터 프레임 csv 파일로 저장

abc7_df.to_csv('abc7_df.csv', encoding='euc-kr')